<a href="https://colab.research.google.com/github/DanielFerreira3900/PremierLeague/blob/main/premier_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from io import StringIO

def download_season(season_code):
    """
    Baixa os dados da temporada indicada pelo código.
    Exemplo: "0203" para 2002/03.
    """
    url = f"https://www.football-data.co.uk/mmz4281/{season_code}/E0.csv"
    print(f"Baixando dados da temporada {season_code} de {url} ...")
    response = requests.get(url)
    if response.status_code == 200:
        try:
            df = pd.read_csv(StringIO(response.text), on_bad_lines='skip')  # pula linhas corrompidas
            return df
        except Exception as e:
            print(f"Erro ao processar CSV da temporada {season_code}: {e}")
            return None
    else:
        print(f"Erro ao baixar dados para a temporada {season_code} (status {response.status_code})")
        return None

def compute_standings(matches):
    """
    A partir do DataFrame 'matches' (com colunas 'HomeTeam', 'AwayTeam',
    'FTHG' e 'FTAG'), calcula a tabela final com:
      - Team, Played, Won, Drawn, Lost, GF, GA, GD e Points.
    """
    teams = pd.concat([matches['HomeTeam'], matches['AwayTeam']]).unique()
    rows = []  # Acumula os dados de cada time
    for team in teams:
        # Resultados jogados em casa e fora
        home = matches[matches['HomeTeam'] == team]
        away = matches[matches['AwayTeam'] == team]

        played = len(home) + len(away)
        home_wins = sum(home['FTHG'] > home['FTAG'])
        home_draws = sum(home['FTHG'] == home['FTAG'])
        home_losses = sum(home['FTHG'] < home['FTAG'])
        away_wins = sum(away['FTAG'] > away['FTHG'])
        away_draws = sum(away['FTAG'] == away['FTHG'])
        away_losses = sum(away['FTAG'] < away['FTHG'])

        won = home_wins + away_wins
        drawn = home_draws + away_draws
        lost = home_losses + away_losses

        GF = home['FTHG'].sum() + away['FTAG'].sum()
        GA = home['FTAG'].sum() + away['FTHG'].sum()
        GD = GF - GA
        points = won * 3 + drawn

        rows.append({
            'Team': team,
            'Played': played,
            'Won': won,
            'Drawn': drawn,
            'Lost': lost,
            'GF': GF,
            'GA': GA,
            'GD': GD,
            'Points': points
        })

    standings = pd.DataFrame(rows)
    standings.sort_values(by=['Points', 'GD', 'GF'], ascending=False, inplace=True)
    standings['Position'] = range(1, len(standings) + 1)
    standings = standings[['Position', 'Team', 'Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', 'Points']]
    return standings

# Dicionário com os códigos das temporadas
seasons = {
    "0203": "2002/03",
    "0304": "2003/04",
    "0405": "2004/05",
    "0506": "2005/06",
    "0607": "2006/07",
    "0708": "2007/08",
    "0809": "2008/09",
    "0910": "2009/10",
    "1011": "2010/11",
    "1112": "2011/12",
    "1213": "2012/13",
    "1314": "2013/14",
    "1415": "2014/15",
    "1516": "2015/16",
    "1617": "2016/17",
    "1718": "2017/18",
    "1819": "2018/19",
    "1920": "2019/20",
    "2021": "2020/21",
    "2122": "2021/22",
    "2223": "2022/23",
    "2324": "2023/24"
}

# Lista para armazenar as tabelas
all_standings = []

# Loop principal para processar cada temporada
for code, season_name in seasons.items():
    matches = download_season(code)
    if matches is not None and {'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'}.issubset(matches.columns):
        standings = compute_standings(matches)
        standings['Season'] = season_name
        all_standings.append(standings)
    else:
        print(f"⚠️ Temporada {season_name} pulada por dados incompletos.")

# Salva o CSV final
if all_standings:
    final_table = pd.concat(all_standings, ignore_index=True)
    cols = ['Season'] + [col for col in final_table.columns if col != 'Season']
    final_table = final_table[cols]
    final_table.to_csv("premier_league_standings_2002_2023.csv", index=False)
    print("✅ CSV gerado com sucesso: premier_league_standings_2002_2023.csv")
else:
    print("❌ Nenhum dado foi coletado.")

Baixando dados da temporada 0203 de https://www.football-data.co.uk/mmz4281/0203/E0.csv ...
Baixando dados da temporada 0304 de https://www.football-data.co.uk/mmz4281/0304/E0.csv ...
Baixando dados da temporada 0405 de https://www.football-data.co.uk/mmz4281/0405/E0.csv ...
Baixando dados da temporada 0506 de https://www.football-data.co.uk/mmz4281/0506/E0.csv ...
Baixando dados da temporada 0607 de https://www.football-data.co.uk/mmz4281/0607/E0.csv ...
Baixando dados da temporada 0708 de https://www.football-data.co.uk/mmz4281/0708/E0.csv ...
Baixando dados da temporada 0809 de https://www.football-data.co.uk/mmz4281/0809/E0.csv ...
Baixando dados da temporada 0910 de https://www.football-data.co.uk/mmz4281/0910/E0.csv ...
Baixando dados da temporada 1011 de https://www.football-data.co.uk/mmz4281/1011/E0.csv ...
Baixando dados da temporada 1112 de https://www.football-data.co.uk/mmz4281/1112/E0.csv ...
Baixando dados da temporada 1213 de https://www.football-data.co.uk/mmz4281/1213

In [2]:
tabela_pl = pd.read_csv('/content/premier_league_standings_2002_2023.csv')
tabela_pl.head()

,Season,Position,Team,Played,Won,Drawn,Lost,GF,GA,GD,Points
0,2002/03,1,Man United,38,25,8,5,74.0,34.0,40.0,83
1,2002/03,2,Arsenal,38,23,9,6,85.0,42.0,43.0,78
2,2002/03,3,Newcastle,38,21,6,11,63.0,48.0,15.0,69
3,2002/03,4,Chelsea,38,19,10,9,68.0,38.0,30.0,67
4,2002/03,5,Liverpool,38,18,10,10,61.0,41.0,20.0,64


In [23]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import warnings

# Suprimir warnings
warnings.filterwarnings("ignore")

# Carregar os dados
tabela_premier = pd.read_csv("premier_league_tabela_2002_2023.csv", index_col= False)
hist = pd.read_csv("dados_premier_league_2024.csv", index_col=False)
# Renomear colunas para português
tabela_premier.rename(columns={
    'Season': 'Temporada',
    'Position': 'Posição',
    'Team': 'Time',
    'Played': 'Jogos',
    'Won': 'Vitórias',
    'Drawn': 'Empates',
    'Lost': 'Derrotas',
    'GF': 'Gols Pró',
    'GA': 'Gols Contra',
    'GD': 'Saldo de Gols',
    'Points': 'Pontos'
}, inplace=True)

# Calcular colunas derivadas
tabela_premier['Pontos por partida'] = (tabela_premier['Pontos'] / tabela_premier['Jogos']).round(2)
tabela_premier['Aproveitamento'] = ((tabela_premier['Pontos'] / (tabela_premier['Jogos'] * 3)) * 100).round(2)

# Função para estilo da tabela
def aplicar_estilos(df):
    def estilos_linhas(s):
        cores = {
            'Zona de Champions': '#023047',
            'Zona de Liga Europa': '#dad7cd',
            'Zona de Rebaixamento': '#c1121f'
        }
        if s.name < 4:
            return [f'background-color: {cores["Zona de Champions"]}; color: white'] * len(s)
        elif 4 <= s.name <= 5:
            return [f'background-color: {cores["Zona de Liga Europa"]}; color: black'] * len(s)
        elif s.name >= len(df) - 3:
            return [f'background-color: {cores["Zona de Rebaixamento"]}; color: white'] * len(s)
        return [''] * len(s)

    return df.style.apply(estilos_linhas, axis=1)

# Configuração do Dashboard
st.set_page_config(page_title="Dashboard Premier League", layout="wide")
st.title('Dashboard Premier League 🏴󠁧󠁢󠁥󠁮󠁧󠁿')

abas = st.tabs(["Classificação", "Valor de Mercado", "Simulação", "Histórico"])

with abas[0]:  # Classificação
    st.header("Classificação por Temporada")

    temporadas = sorted(tabela_premier['Temporada'].unique(), reverse=True)
    temporada_escolhida = st.selectbox("Escolha a temporada:", temporadas, key="temporada_classificacao")

    df_temporada = tabela_premier[tabela_premier['Temporada'] == temporada_escolhida].copy()
    df_temporada.sort_values(by='Posição', inplace=True)

    df_formatado = df_temporada[[
        'Posição', 'Time', 'Jogos', 'Vitórias', 'Empates',
        'Derrotas', 'Gols Pró', 'Gols Contra', 'Saldo de Gols',
        'Pontos', 'Pontos por partida', 'Aproveitamento'
    ]]

    st.markdown(
        aplicar_estilos(df_formatado).hide(axis='index').to_html(),
        unsafe_allow_html=True
    )

with abas[1]:  # Valor de Mercado
    st.header("Análise de Mercado")

    # Processar dados de mercado corrigido
    # Converter valores de mercado para numérico
    hist['Valor_tratado'] = (
        hist['Valor de Mercado (texto)']
        .astype(str)
        .str.replace(r'[^\d.kKmM€]', '', regex=True)
        .replace({'nan': '0', '': '0'})
    )

    # Converter para valores numéricos
    def converter_valor(valor):
        if 'm' in valor.lower():
            return float(valor.lower().replace('m', '').replace('€', '')) * 1_000_000
        elif 'k' in valor.lower():
            return float(valor.lower().replace('k', '').replace('€', '')) * 1_000
        else:
            return float(valor.replace('€', ''))

    hist['Valor_numérico'] = hist['Valor_tratado'].apply(converter_valor)

    # Processar idade
    hist['Idade'] = hist['Idade'].str.replace(',', '').astype(float)

    # Seleção de clube
    clubes = hist['Clube'].unique()
    clube_selecionado = st.selectbox("Selecione um clube:", clubes, key="clube_mercado")

    # Filtrar dados
    df_clube = hist[hist['Clube'] == clube_selecionado]

    # Calcular métricas
    valor_total = df_clube['Valor_numérico'].sum()
    media_idade = df_clube['Idade'].mean().round(1)
    qtd_jogadores = df_clube.shape[0]

    # Exibir métricas
    col1, col2, col3 = st.columns(3)
    col1.metric("Valor Total (€)", f"€ {valor_total:,.2f}")
    col2.metric("Média de Idade", f"{media_idade} anos")
    col3.metric("Jogadores no Elenco", qtd_jogadores)

    # Tabela de jogadores
    st.dataframe(df_clube[['Nome do Jogador', 'Posição', 'Idade', 'Valor_numérico']]
                     .rename(columns={'Valor_numérico': 'Valor (€)'}))

with abas[2]:  # Simulação
    st.header("Simulador de Partidas")

    times = tabela_premier['Time'].unique()
    time_casa = st.selectbox("Time da Casa", times, key="time_casa")
    time_visitante = st.selectbox("Time Visitante", times, key="time_visitante")

    if st.button("Simular Jogo"):
        # Cálculos de probabilidade
        media_gols_casa = (
    tabela_premier[tabela_premier['Time'] == time_casa]['Gols Pró'].values[0] /
    tabela_premier[tabela_premier['Time'] == time_casa]['Jogos'].values[0]
)

        media_gols_visitante = (
    tabela_premier[tabela_premier['Time'] == time_visitante]['Gols Contra'].values[0] /
    tabela_premier[tabela_premier['Time'] == time_visitante]['Jogos'].values[0]
)


        # Simulação Poisson
        gols_casa = np.random.poisson(media_gols_casa)
        gols_visitante = np.random.poisson(media_gols_visitante)

        # Resultado
        resultado = f"{time_casa} {gols_casa} × {gols_visitante} {time_visitante}"
        st.subheader("Resultado da Simulação:")
        st.markdown(
            f"<h1 style='text-align: center; color: #023047;'>{resultado}</h1>",
            unsafe_allow_html=True
        )


with abas[3]:  # Histórico
    st.header("Histórico de Títulos")

    campeoes = tabela_premier[tabela_premier['Posição'] == 1]
    titulos = campeoes['Time'].value_counts().reset_index()
    titulos.columns = ['Time', 'Títulos']

    fig = px.bar(
        titulos,
        x='Time',
        y='Títulos',
        color='Títulos',
        title="Campeões da Premier League",
        color_continuous_scale='Blues'
    )
    st.plotly_chart(fig)

    st.subheader("Selecionar Temporada")
    temporada = st.selectbox("Escolha uma temporada:", tabela_premier['Temporada'].unique())
    st.dataframe(tabela_premier[tabela_premier['Temporada'] == temporada]
                 .sort_values('Posição'))

Overwriting app.py


In [7]:
!pip install -q streamlit pyngrok
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -q ngrok-stable-linux-amd64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [8]:
%%writefile config.toml
[server]
port = 8501
enableCORS = false

Writing config.toml


In [9]:
# Método seguro para usar seu token (não exponha no código)
from getpass import getpass

# Isso pedirá o token de forma segura
NGROK_TOKEN = getpass("2vkAjpgxJNwGYNo6hJBKJbhz5ZM_3uFvuVUyXTHwvBqKrXUFP ")

# Configuração do Streamlit + Ngrok
!streamlit run app.py --server.port 8501 &>/dev/null&

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(8501, "http")
print(f"URL seguro: {public_url}")

2vkAjpgxJNwGYNo6hJBKJbhz5ZM_3uFvuVUyXTHwvBqKrXUFP ··········
URL seguro: NgrokTunnel: "https://29c4-35-231-82-245.ngrok-free.app" -> "http://localhost:8501"


In [10]:
# Iniciar o Streamlit em background
!streamlit run app.py --server.port 8501 --server.fileWatcherType none --browser.serverAddress 0.0.0.0 --server.enableCORS false --server.enableXsrfProtection false &>/dev/null&

# Obter URL público
from pyngrok import ngrok
public_url = ngrok.connect(addr='8501', proto='http')
print(f"Seu dashboard está disponível em: {public_url.public_url}")

Seu dashboard está disponível em: https://5d8b-35-231-82-245.ngrok-free.app
